In [2]:
from skimage import io
import matplotlib.pyplot as plt
import numpy as np 
import napari
import h5py

In [3]:
path = "/Users/thomasathey/Documents/mimlab/mouselight/kolodkin/sriram/files/im0.tif"
im0 = np.squeeze(io.imread(path))
path = "/Users/thomasathey/Documents/mimlab/mouselight/kolodkin/sriram/files/im1.tif"
im1 = np.squeeze(io.imread(path))

v = napari.Viewer()
v.add_image(im0)
v.add_image(im1)

<Image layer 'im1' at 0x17139ccd0>

# scratch

In [14]:
path = "/Users/thomasathey/Documents/mimlab/mouselight/kolodkin/sriram/image_160_Probabilities.h5"
with h5py.File(path, "r") as hf:
    print(hf['exported_data'][:,50,50,50])

[1. 0.]


In [1]:
import napari

v = napari.Viewer()


Gtk-Message: 15:17:26.041: GtkDialog mapped without a transient parent. This is discouraged.


In [2]:
import zarr

z_im = zarr.open("/cis/home/tathey/projects/mouselight/sriram/somez.zarr")
z_im.chunks

(2, 100, 100, 40)

In [6]:
z_im.shape

(2, 6814, 8448, 316)

In [5]:
zarr.open_group("/cis/home/tathey/projects/mouselight/sriram/somez_hier.zarr", "a")

<zarr.hierarchy.Group '/'>